In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"   # disables GPU completely

In [2]:
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Embedding, LSTM, Dense, Dropout,
    Concatenate, RepeatVector, Flatten
)
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [3]:
# ========== CONFIG ==========
DATA_DIR = "../data/training_data"
ARTIFACT_DIR = "artifacts"
MODEL_DIR = "../models"

MOODS = ["happy", "sad", "angry", "quiet"]
SEQ_LEN = 50

NOTE_EMBED_DIM = 128      # reduced for CPU
MOOD_EMBED_DIM = 16
LSTM_UNITS = 256

BATCH_SIZE = 128          # safe for CPU
EPOCHS = 3
# ===========================


In [4]:
os.makedirs(MODEL_DIR, exist_ok=True)

In [5]:
with open(f"{DATA_DIR}/X.pkl", "rb") as f:
    X = pickle.load(f)

with open(f"{DATA_DIR}/y.pkl", "rb") as f:
    y = pickle.load(f)

with open(f"{DATA_DIR}/mood_labels.pkl", "rb") as f:
    moods = pickle.load(f)

with open(f"{ARTIFACT_DIR}/note_to_idx.pkl", "rb") as f:
    note_to_idx = pickle.load(f)

vocab_size = len(note_to_idx)

print("Samples:", X.shape)
print("Vocabulary size:", vocab_size)


Samples: (16716936, 50)
Vocabulary size: 128


In [6]:
note_input = Input(shape=(SEQ_LEN,))
mood_input = Input(shape=(1,))

note_emb = Embedding(vocab_size, NOTE_EMBED_DIM)(note_input)
mood_emb = Embedding(len(MOODS), MOOD_EMBED_DIM)(mood_input)

mood_emb_flat = Flatten()(mood_emb)
mood_rep = RepeatVector(SEQ_LEN)(mood_emb_flat)

x = Concatenate()([note_emb, mood_rep])
x = LSTM(LSTM_UNITS)(x)
x = Dropout(0.3)(x)

output = Dense(vocab_size, activation="softmax")(x)

model = Model([note_input, mood_input], output)

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy"
)

model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 1)]                  0         []                            
                                                                                                  
 embedding_1 (Embedding)     (None, 1, 16)                64        ['input_2[0][0]']             
                                                                                                  
 input_1 (InputLayer)        [(None, 50)]                 0         []                            
                                                                                                  
 flatten (Flatten)           (None, 16)                   0         ['embedding_1[0][0]']         
                                                                                              

In [7]:
checkpoint = ModelCheckpoint(
    filepath=f"{MODEL_DIR}/music_generator_cpu.h5",
    save_best_only=True,
    monitor="loss"
)

early_stop = EarlyStopping(
    monitor="loss",
    patience=2
)

history = model.fit(
    [X, moods],
    y,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[checkpoint, early_stop]
)


Epoch 1/3
130602/130602 [==============================] - 19638s 150ms/step - loss: 1.8816
Epoch 2/3


d:\Study\Ai_Music_Composer\venv\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


130602/130602 [==============================] - 32377s 248ms/step - loss: 1.7787
Epoch 3/3
130602/130602 [==============================] - 34179s 262ms/step - loss: 1.7555


In [8]:
model.save(f"{MODEL_DIR}/music_generator.h5")
print("CPU-trained music generator saved")

CPU-trained music generator saved
